In [ ]:
!unzip Dataset.zip

Archive:  Dataset.zip
   creating: Section1-Chapter3/
replace __MACOSX/._Section1-Chapter3? [y]es, [n]o, [A]ll, [N]one, [r]ename: 

In [ ]:
import pandas as pd
label_list=[]
text_list=[]
file_list=[]
fileDir="Section2-Chapter3"
df=pd.read_csv(fileDir+"/Labels.csv")

for row in df.itertuples(index=True, name='Pandas'):
    fileName=row.FileName+'.asm'
    with open("Section2-Chapter3/"+fileName, encoding='utf-8', errors='ignore') as f:
        content=f.read()
    content.replace(",", " ")
    content.replace('"', " ")
    text_list.append(content)
    file_list.append(fileName)
    label_list.append(row.Label)      

In [ ]:
from tensorflow.keras.preprocessing.text import Tokenizer
num_labels = 3
vocab_size = 11
batch_size = 100

tokenizer = Tokenizer(num_words=vocab_size)
tokenizer.fit_on_texts(text_list)
 
dtm = tokenizer.texts_to_matrix(text_list, mode='count')

df_count=pd.DataFrame(dtm)
df_count['FileName']=pd.Series(file_list)
df_count['Label']=pd.Series(label_list)
df_count.head(15)

In [ ]:

from tensorflow.keras.utils import to_categorical 
from sklearn.model_selection import train_test_split

Y=df_count['Label']
Y_Encoded=to_categorical(Y)

df_count.drop(['Label', 'FileName'], axis=1, inplace=True)
X=df_count


X_Train, X_Test, Y_Train, Y_Test = train_test_split(X, Y_Encoded, test_size=0.20, random_state=40)

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense


count_classes = Y_Test.shape[1]
input_dim=X_Train.shape[1]

model=Sequential()
model.add(Dense(input_dim, activation='relu', input_dim=input_dim))
model.add(Dense(100, activation='relu'))
model.add(Dense(50, activation='relu'))
model.add(Dense(count_classes, activation='softmax'))
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [ ]:
model.fit(X_Train, Y_Train, epochs=100, verbose=0)
predictions=model.predict(X_Test)
scores = model.evaluate(X_Test, Y_Test, verbose=0)
print('Accuracy on test data: {}% \n Error on test data: {}'.format(scores[1], 1 - scores[1]))